In [ ]:
import tensorflow as tf
import gensim
import csv

In [ ]:
# parameters config
flags = tf.flags
FLAGS = flags.FLAGS

flags.DEFINE_string("train_data_path", "../train_data/train.csv", "train data path")
flags.DEFINE_string("dev_data_path", "../train_data/dev.csv", "dev data path")
flags.DEFINE_string("train_tfrecord_path", "../train_data/train_word_id.tf_record", "train data path")
flags.DEFINE_string("dev_tfrecord_path", "../train_data/dev_word_id.tf_record", "dev data path")
flags.DEFINE_integer("n_hidden", 128, "LSTM hidden layer num of features")
flags.DEFINE_integer("num_step", 16, "input data timesteps")
flags.DEFINE_integer("n_classes", 2, "number of classes")
flags.DEFINE_float("learning_rate", 0.01, "learnning rate")
flags.DEFINE_integer("batch_size", 32, "batch size")
flags.DEFINE_integer("max_steps", 4000, "max step,stop condition")
flags.DEFINE_integer("display_step", 1000, "save model steps")
flags.DEFINE_string("train_writer_path", "./logs/train", "train tensorboard save path")
flags.DEFINE_string("dev_writer_path", "./logs/train", "dev tensorboard save path")
flags.DEFINE_string("checkpoint_path", "./logs/checkpoint", "model save path")
# flags.DEFINE_string("glove_path", "./glove.840B.300d/glove.840B.300d.txt", "pre-train embedding model path")
flags.DEFINE_string("glove_path", "../train_data/vocab.txt", "pre-train embedding model path")
flags.DEFINE_integer("embedding_dim", 300, "word embedding dim")
flags.DEFINE_integer("seq_length", 15, "sentence max length")

In [ ]:
# read csv
def _read_csv(input_file):
    """
    read csv file,get data
    :param input_file:
    :return:
    """
    with tf.gfile.Open(input_file, "r") as f:
        reader = csv.reader(f)
        lines = []
        for line in reader:
            lines.append(line)
        return lines[1:]  # remove header

In [ ]:
# change glove to gensim model
def build_embedding_model(glove_file, gensim_file):
    with open(glove_file, 'r', encoding='utf-8') as f:
        num_lines = 0
        for line in f:
            num_lines += 1
    dims = 300
    gensim_first_line = "{} {}".format(num_lines, dims)
    with open(glove_file, 'r', encoding='utf-8') as fin:
        with open(gensim_file, 'w', encoding='utf-8') as fout:
            fout.write(gensim_first_line + '\n')
            for line in fin:
                fout.write(line)

# build vocab
def build_vocab(model_file, data_file, vocab_path):
    # load glove model
    model = gensim.models.KeyedVectors.load_word2vec_format(model_file)
    lines = _read_csv(data_file)
    vocab = []
    for line in lines:
        vocab.extend(sentence_split(line[1], 0))
    vocab = set(vocab)
    with open(vocab_path, 'w', encoding='utf-8') as f:
        for word in vocab:
            if word in model:
                f.write(word + ' ' + ' '.join([str(x) for x in model[word]]) + '\n')
build_vocab(gensim_file, data_file, vocab_path)